# Linearization of Non-Linear Equations

In [ ]:
using Plots, LinearAlgebra
plotly()

## Problem Statement

Consider the following non-linear equation. 

$\displaystyle{u\left( x \right) = x - x^2 + \sqrt\gamma\left( \frac{1}{4\pi}\sin\left( \frac{2\pi x}{\gamma} \right)-
\frac{x}{2\pi}\sin\left( \frac{2\pi x}{\gamma} \right) -\frac{\gamma}{4\pi^2}\cos\left( \frac{2\pi x}{\gamma} \right) + \frac{\gamma}{4\pi^2} \right) }
$

where $\gamma$ is a parameter that can take values from $10^{-1} \text{ to } 10^{-2}$ and $x \in [0,1]$.

In [ ]:
gm = 0.01:0.01:0.1;
x_ini = 0.0:0.01:1.;
γ = gm[1];
x0 = x_ini[1]
u(x, γ=γ) = x-x^2 +√γ*(1/4π * sin(2π*x/γ) - x/2π * sin(2π*x/γ) - γ/4π^2 * cos(2π*x/γ) + γ/4π^2);
du(x) = (1-2x)*(1+ 1/(2*√γ) * cos(2π*x/γ));
uₗ(x, x0=x0) = u(x0) + du(x0)*(x-x0);
x = 0:1e-5:1;
ϵ = 1e-11;
x_star = 0.5;
x_next(xi, γ=γ) = xi + (u(x_star, γ) - u(xi))/du(xi);

In [ ]:
plot(u, x, legend=false)

In [ ]:
u_values = Array{Vector{Float64}}(undef,length(gm), length(x_ini))
residue_values = Array{Vector{Float64}}(undef,length(gm), length(x_ini))
x_values = Array{Vector{Float64}}(undef,length(gm), length(x_ini))
ustar_values = Array{Float64}(undef,length(gm), length(x_ini))
itr_values = Array{Int64}(undef, length(gm), length(x_ini))
for i in 1:length(gm)
    for j in 1:length(x_ini)
        γ = gm[i];
        x0 = x_ini[j];
        xs = [x0]
        xi = last(xs)
        u_star = u(x_star, γ)
        u_L = [uₗ(x_star, xi),]
        residue = [abs(u_star - last(u_L))]
        its = 0
        while last(residue) > ϵ && its < 10^4
            xi = x_next(xi, γ)
            push!(xs, xi)
            push!(u_L, uₗ(x_star, xi))
            push!(residue, abs(u_star - last(u_L)))
            its += 1
        end
        u_values[i, j] = u_L
        residue_values[i,j] = residue
        x_values[i,j] = xs
        ustar_values[i,j] = u_star
        itr_values[i,j] = length(residue)
    end
end

In [ ]:
p1=surface(x_ini, gm, 1 ./itr_values, zlim=(0, 0.15), c=:seaborn_bright,# cgrad(:gnuplot, rev=true),
        xlabel="x₀ value", ylabel="γ value", zlabel="1/Iterations", cbar=false)

In [ ]:
contourf(x_ini, gm, itr_values, zlim=(0, 0.15), c=:seaborn_bright,# cgrad(:gnuplot, rev=true),
        xlabel="x₀ value", ylabel="γ value")

In [ ]:
i = rand(1:length(gm))
j = rand(1:length(x_ini))
itr_values[i,j]

In [ ]:
u_star = ustar_values[i,j]
xs = x_values[i,j]
residue = residue_values[i,j]
γ = gm[i]
u_vals = u_values[i,j]
p1=plot(x->u(x, γ), x, ylim=(0,0.3), xlabel="x", ylabel="u(x)")
plot!(p1, x, fill(u_star, size(x)), lc=:red)
c=0
for xᵢ ∈ xs
    plot!(p1, x->uₗ(x, xᵢ), x, ylim=(0,0.3), xlim=(0,1), legend=false, lc=:black)
    plot!(p1, fill(x_next(xᵢ,γ), size(0:.05:.25)), 0:.05:.25, lc=:orange)
    if length(xs) > 50
        c += 1
        if c > 10
        break
        end
    end
end
p2 = plot(residue, scale=:log10, xlabel="Iteration", ylabel="Residue", legend=false)
plot(p1, p2, layout=(2,1))
p1

In [ ]:
p2